<a href="https://colab.research.google.com/github/dustejuned/nlp/blob/master/fr_2_eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from random import randint
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
import numpy as np

start_char = '\t'
end_char = '\n'
batch_size = 100
epochs = 500
latent_dimensions = 256

def load_corpus(filepath='./fra.txt', samples=3500):
    input_vocab = []
    target_vocab = []
    input_chars = []
    target_chars = []
    with open(filepath, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
        
        for line in lines[: min(samples, len(lines) - 1)]:
            if(len(line.split('\t')) > 1):

                input_text, target_text = line.split('\t')
                
                #append start and end character to target text
                target_text = start_char + target_text + end_char

                input_vocab.append(input_text)
                target_vocab.append(target_text)
                for c in input_text:
                    if c not in input_chars:
                        input_chars.append(c)
                for c in target_text:
                    if c not in target_chars:
                        target_chars.append(c)        

    #covert words to tokens
    num_input_tokens = len(input_chars)
    num_target_tokens = len(target_chars)
    max_input_len = max([len(txt) for txt in input_vocab])
    max_target_len = max([len(txt) for txt in target_vocab])

    input_token_index = dict([(char, i) for i, char in enumerate(input_chars)])
    target_token_index = dict([(char, i) for i, char in enumerate(target_chars)])

    encoder_inputs = np.zeros((len(input_vocab), max_input_len, num_input_tokens), dtype='float32')
    decoder_inputs = np.zeros((len(target_vocab), max_target_len, num_target_tokens), dtype='float32')
    decoder_targets = np.zeros((len(target_vocab), max_target_len, num_target_tokens), dtype='float32')
                         
    for i, (input_text, target_text) in enumerate(zip(input_vocab, target_vocab)):
        for t, c in enumerate(input_text):
            encoder_inputs[i, t, input_token_index[c]] = 1

        for t, c in enumerate(target_text):
            decoder_inputs[i, t, target_token_index[c]] = 1   
            #decoder target will be ahead by on time stamp
            if t > 0:
                decoder_targets[i, t - 1, target_token_index[c]] = 1
    
    return num_input_tokens, num_target_tokens, max_target_len, input_token_index, target_token_index, encoder_inputs, decoder_inputs, decoder_targets

    

def define_model(input_length, output_length, n_units):
    #define encoder
    encoder_inputs = Input(shape=(None, input_length))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, e_state_h, e_state_c = encoder(encoder_inputs)
    #just need encoder states
    e_states = [e_state_h, e_state_c]
    #define decoder
    decoder_inputs = Input(shape=(None, output_length))
    decoder = LSTM(n_units, return_sequences=True, return_state=True)
    #we just need decoder output
    decoded_outputs, _, _ = decoder(decoder_inputs, initial_state=e_states)
    decoder_dense = Dense(output_length, activation='softmax')
    decoded_outputs = decoder_dense(decoded_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoded_outputs)
    #define inference encoder
    inf_encoder = Model(encoder_inputs, e_states)

    #defince inference decoder
    inf_decoder_state_h = Input(shape=(n_units,))
    inf_decoder_state_c = Input(shape=(n_units,))
    inf_decoder_state_inputs = [inf_decoder_state_h, inf_decoder_state_c]
    decoded_outputs, decoder_state_h, decoder_state_c = decoder(decoder_inputs, initial_state=inf_decoder_state_inputs)
    decoder_states = [decoder_state_h, decoder_state_c]
    decoded_outputs = decoder_dense(decoded_outputs)
    inf_decoder = Model([decoder_inputs] + inf_decoder_state_inputs, [decoded_outputs] + decoder_states)
    #return all models
    return model, inf_encoder, inf_decoder

 





num_input_tokens, num_target_tokens, max_target_len, input_token_index, target_token_index, encoder_inputs, decoder_inputs, decoder_targets = load_corpus()
model, inf_encoder, inf_decoder = define_model(num_input_tokens, num_target_tokens, latent_dimensions)
print(model.summary())
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.fit([encoder_inputs, decoder_inputs], decoder_targets, batch_size=batch_size, epochs=epochs, validation_split=0.2)
model.save('fr_2_eng.h5')



Using TensorFlow backend.
W0807 22:25:44.762928 139971175274368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0807 22:25:44.792063 139971175274368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0807 22:25:44.800256 139971175274368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0807 22:25:45.599832 139971175274368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0807 22:25:45.621644 13997117

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 64)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 86)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 328704      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  351232      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

W0807 22:25:47.021238 139971175274368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 2800 samples, validate on 700 samples
Epoch 1/500
2800/2800 [==============================] - 7s 2ms/step - loss: 1.1709 - acc: 0.0578 - val_loss: 1.2807 - val_acc: 0.0515
Epoch 2/500
2800/2800 [==============================] - 2s 858us/step - loss: 1.0484 - acc: 0.0702 - val_loss: 1.2195 - val_acc: 0.0720
Epoch 3/500
2800/2800 [==============================] - 2s 858us/step - loss: 1.0076 - acc: 0.0797 - val_loss: 1.1797 - val_acc: 0.0824
Epoch 4/500
2800/2800 [==============================] - 2s 857us/step - loss: 0.9636 - acc: 0.0969 - val_loss: 1.1334 - val_acc: 0.0899
Epoch 5/500
2800/2800 [==============================] - 2s 845us/step - loss: 0.9107 - acc: 0.1133 - val_loss: 1.0722 - val_acc: 0.1278
Epoch 6/500
2800/2800 [==============================] - 2s 859us/step - loss: 0.8549 - acc: 0.1280 - val_loss: 0.9988 - val_acc: 0.1419
Epoch 7/500
2800/2800 [==============================] - 2s 845us/step - loss: 0.8060 - acc: 0.1398 - val_loss: 0.9685 - val_acc: 0.1

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [0]:
def predict_model(inf_encoder, inf_decoder, input_seq, num_target_tokens, start_idx, reverse_target_char_index, max_target_seq):
    states = inf_encoder.predict(input_seq)
    target = np.zeros((1, 1, num_target_tokens), dtype='float32')
    target[0, 0, start_idx] = 1
    predicted_seq = ''
    stop = False
    while not stop:
        output, h, c = inf_decoder.predict([target] + states)
        output_token = np.argmax(output[0, -1, :])
        predicted_seq += reverse_target_char_index[output_token]

        if(reverse_target_char_index[output_token] == '\n' or len(predicted_seq) > max_target_seq):
            stop = True

        states = [h, c]
        target[0, 0, output_token] = 1

    return predicted_seq

In [0]:
for seq_index in range(100):
  
  test_input_seq = encoder_inputs[seq_index:seq_index + 1]

  start_idx = target_token_index["\t"]
  reverse_target_char_index = dict(
      (i, char) for char, i in target_token_index.items())
  reverse_input_char_index = dict(
      (i, char) for char, i in input_token_index.items())  
  test_input = ''
  for token in test_input_seq[0]:
    if(np.sum(token) > 0):
      sample_token = np.argmax(token)
      test_input += reverse_input_char_index[sample_token]    
  predicted_seq = predict_model(inf_encoder, inf_decoder, test_input_seq, num_target_tokens, start_idx, reverse_target_char_index, max_target_len)

  print('Input: %s, Predicted: %s' % (test_input, predicted_seq))

Input: Go., Predicted: Va !

Input: Hi., Predicted: Salut. 

Input: Hi., Predicted: Salut. 

Input: Run!, Predicted: Coomepprssssssssssssssssssssssssss
Input: Run!, Predicted: Coomepprssssssssssssssssssssssssss
Input: Who?, Predicted: Qui ?

Input: Wow!, Predicted: Ça alaaaaaasa aaa aa aa a a faaaaa
Input: Fire!, Predicted: Au lde ddffreffffeefeeererrrrrrrrd
Input: Help!, Predicted: À l'aate                          
Input: Jump., Predicted: Sautte. 

Input: Stop!, Predicted: Ça su  ffas  t raaaiaiaiaiaiaaaaaa
Input: Stop!, Predicted: Ça su  ffas  t raaaiaiaiaiaiaaaaaa
Input: Stop!, Predicted: Ça su  ffas  t raaaiaiaiaiaiaaaaaa
Input: Wait!, Predicted: Attdeddz ------olllllll  ll    lll
Input: Wait!, Predicted: Attdeddz ------olllllll  ll    lll
Input: Go on., Predicted: Pourrssuez pvpezzzuuuuuuuuuusassss
Input: Go on., Predicted: Pourrssuez pvpezzzuuuuuuuuuusassss
Input: Go on., Predicted: Pourrssuez pvpezzzuuuuuuuuuusassss
Input: Hello!, Predicted: Salut e  ! 

Input: Hello!, Predict